# Face Detection

In [2]:
import cv2

# Load the face detection classifier
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Read the video stream
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        # Draw rectangles around detected faces
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

    cv2.imshow('Face Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


# Attention Detection

In [3]:
import cv2
import dlib
from scipy.spatial import distance as dist
import numpy as np

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])

    C = dist.euclidean(eye[0], eye[3])

    ear = (A + B) / (2.0 * C)
    return ear

EAR_THRESHOLD = 0.2
CONSECUTIVE_FRAMES_THRESHOLD = 3  
HEAD_ROTATION_THRESHOLD = 5

STATIC_ATTENTION_ANGLE_THRESHOLD = 0 
MAX_ATTENTION_PROBABILITY = 1.0  
MIN_ATTENTION_PROBABILITY = 0.85  
frame_counter = 0
blink_counter = 0
head_rotation = "Forward"  
attention_probability = MAX_ATTENTION_PROBABILITY  
cap = cv2.VideoCapture('cam1.mp4')

while True:
    ret, frame = cap.read()
    if not ret:
        break
        
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    if len(faces) == 0:

        cv2.putText(frame, 'CRITICAL WARNING: No Face Detected!', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)


    for (x, y, w, h) in faces:
        roi_gray = gray[y:y + h, x:x + w]
        landmarks = predictor(roi_gray, dlib.rectangle(0, 0, w, h))
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        x_offset = x  
        y_offset = y 

        left_eye_center = np.mean([(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)], axis=0)
        right_eye_center = np.mean([(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)], axis=0)
        nose_tip = (landmarks.part(30).x, landmarks.part(30).y)

        left_eye_to_nose = dist.euclidean(left_eye_center, nose_tip)
        right_eye_to_nose = dist.euclidean(right_eye_center, nose_tip)

        if left_eye_to_nose > right_eye_to_nose + HEAD_ROTATION_THRESHOLD:
            head_rotation = "Left"
        elif right_eye_to_nose > left_eye_to_nose + HEAD_ROTATION_THRESHOLD:
            head_rotation = "Right"
        else:
            head_rotation = "Forward"

        head_rotation_angle = left_eye_to_nose - right_eye_to_nose

        if abs(head_rotation_angle) <= STATIC_ATTENTION_ANGLE_THRESHOLD:
            attention_probability = MAX_ATTENTION_PROBABILITY
        else:
            attention_probability = MAX_ATTENTION_PROBABILITY - (abs(head_rotation_angle) - STATIC_ATTENTION_ANGLE_THRESHOLD) / (180 - STATIC_ATTENTION_ANGLE_THRESHOLD)

        for i in range(68):
            x, y = landmarks.part(i).x, landmarks.part(i).y
            cv2.circle(frame, (x + x_offset, y + y_offset), 2, (0, 0, 255), -1)

        left_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)])
        right_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)])

        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)

        avg_ear = (left_ear + right_ear) / 2.0

        if avg_ear < EAR_THRESHOLD:
            frame_counter += 1
        else:
            if frame_counter >= CONSECUTIVE_FRAMES_THRESHOLD:
                blink_counter += 1
            frame_counter = 0

        if attention_probability < MIN_ATTENTION_PROBABILITY:
            cv2.putText(frame, 'WARNING: Low Attention!', (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

    cv2.putText(frame, f'Eye Countdown: {blink_counter}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, f'Head Rotation: {head_rotation}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.putText(frame, f'Attention Probability: {attention_probability:.2f}', (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv2.imshow('Attention Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
